### This  file is specific to preprocessing and first-round data cleaning requirements. Take the new JSON files generated from here and use it separately for the purposes of exploration and modelling

In [ ]:
import numpy as np
import scipy
import random
import urllib
import matplotlib.pyplot as plt
import json

In [ ]:
random.seed(0) #set an initial seed so that we can reproduce consistently

In [ ]:
def readJSONfromFile(fname):
    yelp_data = []
    with open(fname) as f:
        for line in f:
            yelp_data.append(json.loads(line))
    return yelp_data

In [ ]:
IuVegas = readJSONfromFile('yelp_user_IuVegas.json')
UiVegas = readJSONfromFile('yelp_user_UiVegas.json')

In [ ]:
IuVegas = IuVegas[0]
UiVegas = UiVegas[0]

In [ ]:
I = readJSONfromFile('data/yelp_restaurant_data.json')

In [ ]:
I = I[0]

In [ ]:
#I[:10]

In [ ]:
VegasUser = []
for user in IuVegas.keys():
    if len(IuVegas[user]) >= 10:
        VegasUser.append(user)

In [ ]:
len(IuVegas[VegasUser[0]])


In [ ]:
len(IuVegas)

In [ ]:
S_ui_test = {}

In [ ]:
for user in VegasUser:
    temp = random.choice(IuVegas[user])
    IuVegas[user].remove(temp)
    S_ui_test[user] = temp

In [ ]:
len(IuVegas[VegasUser[0]])

In [ ]:
len(S_ui_test)

In [ ]:
len(VegasUser)

In [ ]:
VegasRest = UiVegas.keys()

In [ ]:
len(VegasUser)

In [ ]:
Ui = UiVegas
UiVegas = {}

In [ ]:
for rest in VegasRest:
    UiVegas[rest] = []

In [ ]:
for rest in VegasRest:
    for u in Ui[rest]:
        if u in VegasUser:
            UiVegas[rest].append(u)

In [ ]:
len(VegasUser)

In [ ]:
IV = readJSONfromFile('yelp_vegas_reviewrating_data.json')

In [ ]:
IVegas=IV[0]

In [ ]:
len(IVegas)

In [ ]:
IVegas[0]

In [ ]:
Index_u = {}
Index_r = {}
count = 0
for user in VegasUser:
    Index_u[user] = count
    count += 1
count = 0
for rest in VegasRest:
    Index_r[rest] = count
    count += 1

In [ ]:
M = np.zeros((len(VegasRest),len(VegasUser)),dtype=np.float32)


In [ ]:
M.shape

In [ ]:
R = {}

In [ ]:
for i in IVegas:
    R[(i['user_id'],i['business_id'])] = i['stars']

In [ ]:
for user in VegasUser:
    for rest in VegasRest:
        if R.has_key((user,rest)):
            M[Index_r[rest]][Index_u[user]] = R[(user,rest)]

In [ ]:
for user in S_ui_test.keys():
    M[Index_r[S_ui_test[user]]][Index_u[user]] = 0

In [ ]:
M.shape[0]

In [ ]:
for i in range(M.shape[0]):
    if np.linalg.norm(M[i]) != 0:
        M[i] = M[i]-np.mean(M[i])
        M[i] = M[i]*1.0/np.linalg.norm(M[i])

In [ ]:
Similarity_rr = np.matmul(M,M.T )


In [ ]:
def rating(user,b):
    su1 = 0
    su2 = 0
    for i in IuVegas[user]:
        su1 += Similarity_rr[Index_r[i]][Index_r[b]]*R[(user,i)]
        su2 += np.abs(Similarity_rr[Index_r[i]][Index_r[b]])
    if su2 == 0:
        return 0
    return su1*1.0/su2

In [ ]:
total = 0
count = 0
for user in VegasUser:
    count += 1
    print count
    su = 0
    i = S_ui_test[user]
    jAbsent = list(set(VegasRest) - set(IuVegas[user]) - set(S_ui_test))
    for j in jAbsent:
#         print np.dot(Gamma_u[user],Gamma_r[i]),np.dot(Gamma_u[user],Gamma_r[j]),j
        if rating(user,i) > rating(user,j):
            su += 1
    total += su*1.0/(len(jAbsent))
total = total*1.0/(len(VegasUser))
print total

0.619692596547 for 10 iterations

In [ ]:
iterations = 1000000000

In [ ]:
len(val_l)

In [ ]:
plt.plot(range(50),val_l)
plt.show()

In [ ]:
plt.plot(range(100),train_l)
plt.show()

In [ ]:
total = 0
count = 0
for user in VegasUser:
    count += 1
    print count
    su = 0
    i = S_ui_test[user]
    jAbsent = []
    if S_uj_train.has_key(user):
        jAbsent = list(set(VegasRest) - set(S_uj_train[user]) - set(S_ui_test))
    else:
        jAbsent = list(set(VegasRest) - set(S_ui_test))
    
    for j in jAbsent:
        if np.dot(Gamma_u[user],Gamma_r[i]) > np.dot(Gamma_u[user],Gamma_r[j]):
            su += 1
    total += su*1.0/(len(jAbsent))
total = total*1.0/(len(VegasUser))
print total

0.01, Sai's method, 10**-4, 10000 iter, K = 10, AUC = 0.515965891654

### Baseline

In [ ]:
UiVegas_b = readJSONfromFile('yelp_user_UiVegas.json') 
UiVegas_b = UiVegas_b[0]

In [ ]:
Ui_len= {}
for rest in UiVegas_b:
    Ui_len[rest]= len(UiVegas_b[rest])


In [ ]:
total = 0
count = 0
for user in VegasUser:
    #count += 1
    #print count
    su = 0
    i = S_ui_test[user]
    xui = Ui_len[i]
    jAbsent = []
    if S_uj_train.has_key(user):
        jAbsent = list(set(VegasRest) - set(S_uj_train[user]) - set(S_ui_test))
    else:
        jAbsent = list(set(VegasRest) - set(S_ui_test))
    
    for j in jAbsent:
        xuj = Ui_len[j]
        if xui > xuj :
            su += 1
    total += su*1.0/(len(jAbsent))
total = total*1.0/(len(VegasUser))


In [ ]:
print("Baseline AUC ={}".format(total))